In [25]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
import re
from serpapi import GoogleSearch
import wikipedia
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import re
from jupyter_client import KernelManager
import ast
import operator
load_dotenv()

True

In [26]:
from typing import List, Optional, Any,Annotated
from pydantic import BaseModel, Field

class GraphState(BaseModel):
    user_query: str = Field(..., description="The original input question or request from the user.")
    tasks: List[str] = Field(default_factory=list, description="List of sub-tasks generated from the user query.")
    tool: List[str] = Field(default_factory=list, description="List of the tools to be used for the particular task.")
    current_task_index: int = Field(default=0, description="Index of the task currently being processed.")
    final_output: Annotated[List[str], operator.add] = Field(default=None, description="Final result to be returned to the user.")
    grade:int 


In [27]:


llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')


    

In [113]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    temperature = 0.7
)

In [114]:
# state = GraphState(user_query="I want to know ideas for my startup")

In [115]:
# state.tasks

In [116]:


def run_in_jupyter_kernel(code: str) -> str:
    """
    Executes Python code in a temporary Jupyter kernel and returns the output.
    
    Args:
        code (str): Python code to execute.
        
    Returns:
        str: Output from the execution (stdout or error).
    """
    # Start a kernel
    print('kernel')
    km = KernelManager()
    km.start_kernel()
    kc = km.client()
    kc.start_channels()

    try:
        # Send the code for execution
        kc.execute(code)

        # Collect output
        output = ""
        while True:
            msg = kc.get_iopub_msg()
            msg_type = msg['msg_type']

            if msg_type == 'stream':
                output += msg['content']['text']
            elif msg_type == 'execute_result':
                output += str(msg['content']['data']['text/plain'])
            elif msg_type == 'error':
                output += '\n'.join(msg['content']['traceback'])
            elif msg_type == 'status' and msg['content']['execution_state'] == 'idle':
                break

        return output.strip()

    finally:
        # Clean up kernel
        kc.stop_channels()
        km.shutdown_kernel()


In [117]:

def extract_code_from_aimessage(message) -> str:
    """
    Extracts and cleans Python code from an AIMessage with markdown-style triple backticks.
    """
    # Extract the code block using regex
    match = re.search(r"```python(.*?)```", message.content, re.DOTALL)
    if match:
        code = match.group(1).strip()
        return code
    else:
        return message.content.strip()  # Fallback if no markdown code block found

In [28]:


code_prompt = '''You are a Python developer helping solve a sub-task as part of a larger user query.

Your job is to write clean, functional Python code that directly solves the following sub-task:

Sub-task: {sub_task}

Constraints:
- Only output executable Python code (no explanations).
- Assume standard Python libraries (like pandas, numpy, matplotlib, requests, etc.) are available.
- If data is needed, generate a mock dataset inline or simulate it appropriately.
- Include print statements for key outputs.
- Code should be suitable for running in a Jupyter notebook cell.

Now, write the code.
'''


def code_generation(state:GraphState):
    '''Tool to generate code and also to execute it in the jupyter kernel to verify the output'''
    task = state.tasks[state.current_task_index]
    prompt = ChatPromptTemplate.from_template(template=code_prompt)
    chain = prompt | llm 
    code = chain.invoke({'sub_task':task})
    code = extract_code_from_aimessage(code)
    output = run_in_jupyter_kernel(code)
    return {'final_output':state.final_output+[f"{task} \n and the code is {code} \n The output of the code generated is {output}"]}




    

In [119]:
# state = GraphState(user_query = "writing code",final_output = [],tasks = ['write a code to print first 10 fibonacci numbers'],current_task_index=0)
# op = code_generation(state)

In [120]:

tavily = TavilySearchResults()

def websearch(state:GraphState):
    '''Tool to perform web search'''
    print('search')
    task = state.tasks[state.current_task_index]
    results = tavily.invoke(task)
    print(results)
    return {'final_output':state.final_output+[f"The output of the code generated is {results[0]['content']}"]}


In [121]:
# state = GraphState(user_query="Nothing",current_task_index=0,tasks = ['Code for fibonacci number is ?'],final_output=[])
# op = websearch(state)

In [122]:
# print(op['final_output'][0])

In [123]:


def wiki_explainer_tool(state:GraphState):
    """Explains a topic using Wikipedia summary."""
    print("wiki")
    task = state.tasks[state.current_task_index]
    try:
        out = wikipedia.summary(task, sentences=5)
    except Exception as e:
       return {"final_output":state.final_output}
    return {"final_output":state.final_output+[f'The wikipedia says :: {out}']}

In [124]:
# state

In [125]:
# wiki_explainer_tool(state)

In [126]:


def youtube_search_serpapi(state: GraphState):
    print("yt")
    params = {
        "engine": "youtube",
        "search_query": state.tasks[state.current_task_index],
        "api_key":"0900f7a2f830083d80385bc46c1ff1f1e6626da3f568de640fe13759e9655450"
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    videos = results.get("video_results", [])[:5]
    print(videos)
    formatted_results = [
        f"{i + 1}. {v.get('title')}\n   {v.get('link')}"
        for i, v in enumerate(videos)
    ]

    return { 'final_output': state.final_output + ["Related youtube videso" ] +formatted_results}



In [127]:
# state = GraphState(user_query="Nothing",current_task_index=0,tasks = ['Code for fibonacci number is ?'],final_output=[])
# op = youtube_search_serpapi(state)


In [128]:
# op

In [129]:
PLANNING_PROMPT = """
You are an expert task planner AI that decomposes complex goals into minimal, actionable sub-tasks for an autonomous agent system.

You have access to the following tools:
- "web_search": Use this to search the internet for current or broad information.
- "code_writer": Use this to generate or edit code.
- "code_executor": Use this to run and test code snippets.
- "wikipedia_search": Use this to retrieve structured knowledge from Wikipedia.
- "youtube_search": Use this to find explanatory or educational videos.
- "text_generator": Use this to write structured content, summaries, or descriptive text.

Given the following user query:

{user_query}

Your job is to:
- Understand the user's objective.
- Break it down into the **smallest possible tasks**, ideally atomic steps that can be executed independently.
- Ensure the tasks are written clearly, with no ambiguity.
- Each task should describe **what** to do, not **how** to do it.
- Assign the most appropriate tool to each task from the list provided.
- Maintain a logical execution order from beginning to end.

Just Return the output as a Python list of dictionaries, each containing:
- "task": A clear, atomic action to be done
- "tool": The most relevant tool to use for this task
dont add anything before or after that


Only return the list — do not include any explanation or extra text.
"""


In [130]:

def break_task(state: GraphState) :
    print("breaking task")
    planning_prompt = ChatPromptTemplate.from_template(PLANNING_PROMPT)
    planning_chain= planning_prompt | llm  


    tasks = planning_chain.invoke({"user_query": state.user_query})

    print("Planned Tasks:", tasks)  
    task_string = tasks.content
    print(task_string)
    match = re.search(r"\[.*\]", task_string, re.DOTALL)
    if not match:
        raise ValueError("No valid task list found in the string.")
    
    task_list_str = match.group(0)
    l =  ast.literal_eval(task_list_str)
    task = []
    tool = []
    for i in list(l):
        task.append(i['task'])
        tool.append(i['tool'])
    return {'tasks':task  , 'tool':tool,'grade':state.grade+1}

    

In [131]:
tools = [websearch,wiki_explainer_tool,code_generation,run_in_jupyter_kernel]

llm = llm.bind_tools(tools = tools)

In [132]:
# state = GraphState(user_query="Writing a code for fibonacci sequence",current_task_index=0,tasks = [],final_output=[],task_outputs=[])
# tasks = break_task(state)

In [133]:
# tasks

In [134]:

GRADE_PROMPT = '''
"Given the user's query: {userquery} and the AI's response: {final_answer}, evaluate the quality of the answer based on the following criteria:

Relevance: Does the answer directly address the user's question? Is it on-topic?

Clarity: Is the answer easy to understand? Are there any ambiguous or unclear sections?

Accuracy: Does the answer provide factually correct and reliable information?

Completeness: Does the answer cover all aspects of the user's query, or are there missing details?

Tone and Professionalism: Does the response maintain a tone appropriate for the context, such as being polite, formal, or informative?

Provide a score from 1 to 10 for each of these criteria, with 1 being very poor and 10 being excellent. Then, calculate an overall average score for the answer. If the score is below 6, suggest improvements to the AI’s response."

Just and just give me a integer number nothing else 
'''



In [135]:
def grading(state:GraphState):
    print("grading")
    prompt = ChatPromptTemplate.from_template(template = GRADE_PROMPT )
    chain = prompt | llm
    grade = chain.invoke({'user_query':state.user_query,'final_answer':state.final_output})
    return {'grade':grade}


In [ ]:


def router_node(state):
    print("router node")
    task_names = [
        "code_generation",
        "run_in_jupyter_kernel"
    ]
    
    
    current_task = task_names[state.current_task_index]
    return current_task

def continue_or_not(state:GraphState):
    print("continue or not ",state.grade)
    if state.grade > 1:
        return END  
    else:
        return "break_task"  


builder = StateGraph(GraphState)


builder.add_node(websearch)
builder.add_node(wiki_explainer_tool)
builder.add_node(code_generation)
builder.add_node(run_in_jupyter_kernel)
builder.add_node(youtube_search_serpapi)
builder.add_node(grading)
builder.add_node(break_task)


builder.add_edge(START, "break_task")  
builder.add_conditional_edges("break_task", router_node)  
builder.add_edge("break_task","youtube_search_serpapi")
builder.add_edge("break_task","wiki_explainer_tool")
builder.add_edge("break_task","websearch")
builder.add_edge("websearch", "grading")
builder.add_edge("wiki_explainer_tool", "grading")
builder.add_edge("youtube_search_serpapi", "grading")
builder.add_edge("code_generation", "grading")
builder.add_edge("run_in_jupyter_kernel", "grading")
builder.add_conditional_edges("grading", continue_or_not)  





In [137]:
graph = builder.compile()

In [138]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	websearch(websearch)
	wiki_explainer_tool(wiki_explainer_tool)
	code_generation(code_generation)
	run_in_jupyter_kernel(run_in_jupyter_kernel)
	youtube_search_serpapi(youtube_search_serpapi)
	grading(grading)
	break_task(break_task)
	__end__(<p>__end__</p>)
	__start__ --> break_task;
	break_task --> wiki_explainer_tool;
	break_task --> youtube_search_serpapi;
	code_generation --> grading;
	run_in_jupyter_kernel --> grading;
	wiki_explainer_tool --> grading;
	youtube_search_serpapi --> grading;
	break_task -.-> websearch;
	break_task -.-> wiki_explainer_tool;
	break_task -.-> code_generation;
	break_task -.-> run_in_jupyter_kernel;
	break_task -.-> youtube_search_serpapi;
	break_task -.-> grading;
	break_task -.-> __end__;
	grading -.-> websearch;
	grading -.-> wiki_explainer_tool;
	grading -.-> code_generation;
	grading -.-> run_in_jupyter_kernel;
	grading -.-> youtube_search_serpapi;
	grad

In [139]:
# graph

In [140]:
# state = GraphState(user_query="Writing a code for fibonacci sequence",current_task_index=0,tasks = [],final_output=[],task_outputs=[],grade = 0)
# # state =graph.invoke(state)

In [141]:
# state

In [142]:
# for i in state['final_output']:
#     display(Markdown(i))


In [ ]:
state = GraphState(user_query="Writing a code for fibonacci sequence",current_task_index=0,tasks = [],final_output=[],task_outputs=[],grade = 0)
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
state = graph.invoke(state,config=config)

In [2]:
state

{'user_query': 'Writing a code for fibonacci sequence',
 'tasks': ['Generate Python code for calculating the Fibonacci sequence.',
  'Test the generated code with a few inputs to verify its correctness.',
  'Check if the code handles edge cases (e.g., negative inputs, zero input).',
  'Optimize the code for efficiency if necessary (e.g., using dynamic programming or memoization).',
  'Add comments and documentation to improve code readability.',
  'If the code needs further improvements, iterate on steps 2-4.'],
 'tool': ['code_writer',
  'code_executor',
  'code_executor',
  'code_writer',
  'code_writer',
  'code_writer'],
 'current_task_index': 0,
 'final_output': ['The output of the code generated is Your first approach to generating the Fibonacci sequence will use a Python class and recursion. An advantage of using the class over the memoized recursive function you saw before is that a class keeps state and behavior (encapsulation) together within the same object. In the function 

In [10]:
from IPython.display import display,Markdown
for i in state['final_output']:
    display(Markdown(i))


The output of the code generated is A Fibonacci sequence is the integer sequence of 0, 1, 1, 2, 3, 5, 8....
The first two terms are 0 and 1. All other terms are obtained by adding the preceding two terms. This means to say the nth term is the sum of (n-1)th and (n-2)th term.
Source Code
```
Program to display the Fibonacci sequence up to n-th term
nterms = int(input("How many terms? "))
first two terms
n1, n2 = 0, 1
count = 0
check if the number of terms is valid
if nterms <= 0:
   print("Please enter a positive integer") [...] if there is only one term, return n1
elif nterms == 1:
   print("Fibonacci sequence upto",nterms,":")
   print(n1)
generate fibonacci sequence
else:
   print("Fibonacci sequence:")
   while count < nterms:
       print(n1)
       nth = n1 + n2
       # update values
       n1 = n2
       n2 = nth
       count += 1
```
Output
How many terms? 7
Fibonacci sequence:
0
1
1
2
3
5
8
Here, we store the number of terms in nterms. We initialize the first term to 0 and the second term to 1. [...] If the number of terms is more than 2, we use a while loop to find the next term in the sequence by adding the preceding two terms. We then interchange the variables (update it) and continue on with the process.
You can also print the Fibonacci sequence using recursion.
Before we wrap up, let's put your understanding of this example to the test! Can you solve the following challenge?
Challenge:
Write a function to get the Fibonacci sequence less than a given number.

The wikipedia says :: An algorithm is fundamentally a set of rules or defined procedures that is typically designed and used to solve a specific problem or a broad set of problems. 
Broadly, algorithms define process(es), sets of rules, or methodologies that are to be followed in calculations, data processing, data mining, pattern recognition, automated reasoning or other problem-solving operations. With the increasing automation of services, more and more decisions are being made by algorithms. Some general examples are; risk assessments, anticipatory policing, and pattern recognition technology.
The following is a list of well-known algorithms along with one-line descriptions for each.

The YouTube videos suggested are:

1. #38 Python Tutorial for Beginners | Fibonacci Sequence
   https://www.youtube.com/watch?v=7Sv4NmvdHcw

2. Generate the Fibonacci Sequence With Python
   https://www.youtube.com/watch?v=I_Giq4-2Pn8

3. Fibonacci Series in Python
   https://www.youtube.com/watch?v=Ib1bSQdXBZ0

4. Fibonacci series using recursive function in Python
   https://www.youtube.com/watch?v=Es5mHeflNCA

5. Python program to print fibonacci series upto n terms using for loop
   https://www.youtube.com/watch?v=K9Qr8bL8_UU

In [8]:
state = {'user_query': 'Writing a code for fibonacci sequence', 'tasks': ['Generate Python code for calculating the Fibonacci sequence up to a given number of terms.', 'Test the generated code with a small number of terms (e.g., 10) to verify its correctness.', 'Optimize the code for efficiency if necessary (e.g., using dynamic programming or memoization).', 'Thoroughly test the optimized code with a larger number of terms to ensure its accuracy and performance.', 'Add comprehensive comments and documentation to the code.', 'If needed, search for examples of Fibonacci sequence code implementations online for comparison and learning.'], 'tool': ['code_writer', 'code_executor', 'code_writer', 'code_executor', 'code_writer', 'web_search'], 'current_task_index': 0, 'final_output': ['The output of the code generated is A Fibonacci sequence is the integer sequence of 0, 1, 1, 2, 3, 5, 8....\nThe first two terms are 0 and 1. All other terms are obtained by adding the preceding two terms. This means to say the nth term is the sum of (n-1)th and (n-2)th term.\nSource Code\n```\nProgram to display the Fibonacci sequence up to n-th term\nnterms = int(input("How many terms? "))\nfirst two terms\nn1, n2 = 0, 1\ncount = 0\ncheck if the number of terms is valid\nif nterms <= 0:\n   print("Please enter a positive integer") [...] if there is only one term, return n1\nelif nterms == 1:\n   print("Fibonacci sequence upto",nterms,":")\n   print(n1)\ngenerate fibonacci sequence\nelse:\n   print("Fibonacci sequence:")\n   while count < nterms:\n       print(n1)\n       nth = n1 + n2\n       # update values\n       n1 = n2\n       n2 = nth\n       count += 1\n```\nOutput\nHow many terms? 7\nFibonacci sequence:\n0\n1\n1\n2\n3\n5\n8\nHere, we store the number of terms in nterms. We initialize the first term to 0 and the second term to 1. [...] If the number of terms is more than 2, we use a while loop to find the next term in the sequence by adding the preceding two terms. We then interchange the variables (update it) and continue on with the process.\nYou can also print the Fibonacci sequence using recursion.\nBefore we wrap up, let\'s put your understanding of this example to the test! Can you solve the following challenge?\nChallenge:\nWrite a function to get the Fibonacci sequence less than a given number.', 'The wikipedia says :: An algorithm is fundamentally a set of rules or defined procedures that is typically designed and used to solve a specific problem or a broad set of problems. \nBroadly, algorithms define process(es), sets of rules, or methodologies that are to be followed in calculations, data processing, data mining, pattern recognition, automated reasoning or other problem-solving operations. With the increasing automation of services, more and more decisions are being made by algorithms. Some general examples are; risk assessments, anticipatory policing, and pattern recognition technology.\nThe following is a list of well-known algorithms along with one-line descriptions for each.', 'The YouTube videos suggested are:', '1. #38 Python Tutorial for Beginners | Fibonacci Sequence\n   https://www.youtube.com/watch?v=7Sv4NmvdHcw', '2. Generate the Fibonacci Sequence With Python\n   https://www.youtube.com/watch?v=I_Giq4-2Pn8', '3. Fibonacci Series in Python\n   https://www.youtube.com/watch?v=Ib1bSQdXBZ0', '4. Fibonacci series using recursive function in Python\n   https://www.youtube.com/watch?v=Es5mHeflNCA', '5. Python program to print fibonacci series upto n terms using for loop\n   https://www.youtube.com/watch?v=K9Qr8bL8_UU'], 'grade': 3}
               

In [9]:
state['final_output']

['The output of the code generated is A Fibonacci sequence is the integer sequence of 0, 1, 1, 2, 3, 5, 8....\nThe first two terms are 0 and 1. All other terms are obtained by adding the preceding two terms. This means to say the nth term is the sum of (n-1)th and (n-2)th term.\nSource Code\n```\nProgram to display the Fibonacci sequence up to n-th term\nnterms = int(input("How many terms? "))\nfirst two terms\nn1, n2 = 0, 1\ncount = 0\ncheck if the number of terms is valid\nif nterms <= 0:\n   print("Please enter a positive integer") [...] if there is only one term, return n1\nelif nterms == 1:\n   print("Fibonacci sequence upto",nterms,":")\n   print(n1)\ngenerate fibonacci sequence\nelse:\n   print("Fibonacci sequence:")\n   while count < nterms:\n       print(n1)\n       nth = n1 + n2\n       # update values\n       n1 = n2\n       n2 = nth\n       count += 1\n```\nOutput\nHow many terms? 7\nFibonacci sequence:\n0\n1\n1\n2\n3\n5\n8\nHere, we store the number of terms in nterms. We 

In [ ]:

from IPython.display import display,Markdown
for i in state['final_output']:
    display(Markdown(i))


The output of the code generated is A Fibonacci sequence is the integer sequence of 0, 1, 1, 2, 3, 5, 8....
The first two terms are 0 and 1. All other terms are obtained by adding the preceding two terms. This means to say the nth term is the sum of (n-1)th and (n-2)th term.
Source Code
```
Program to display the Fibonacci sequence up to n-th term
nterms = int(input("How many terms? "))
first two terms
n1, n2 = 0, 1
count = 0
check if the number of terms is valid
if nterms <= 0:
   print("Please enter a positive integer") [...] if there is only one term, return n1
elif nterms == 1:
   print("Fibonacci sequence upto",nterms,":")
   print(n1)
generate fibonacci sequence
else:
   print("Fibonacci sequence:")
   while count < nterms:
       print(n1)
       nth = n1 + n2
       # update values
       n1 = n2
       n2 = nth
       count += 1
```
Output
How many terms? 7
Fibonacci sequence:
0
1
1
2
3
5
8
Here, we store the number of terms in nterms. We initialize the first term to 0 and the second term to 1. [...] If the number of terms is more than 2, we use a while loop to find the next term in the sequence by adding the preceding two terms. We then interchange the variables (update it) and continue on with the process.
You can also print the Fibonacci sequence using recursion.
Before we wrap up, let's put your understanding of this example to the test! Can you solve the following challenge?
Challenge:
Write a function to get the Fibonacci sequence less than a given number.

The wikipedia says :: An algorithm is fundamentally a set of rules or defined procedures that is typically designed and used to solve a specific problem or a broad set of problems. 
Broadly, algorithms define process(es), sets of rules, or methodologies that are to be followed in calculations, data processing, data mining, pattern recognition, automated reasoning or other problem-solving operations. With the increasing automation of services, more and more decisions are being made by algorithms. Some general examples are; risk assessments, anticipatory policing, and pattern recognition technology.
The following is a list of well-known algorithms along with one-line descriptions for each.

The YouTube videos suggested are:

1. #38 Python Tutorial for Beginners | Fibonacci Sequence
   https://www.youtube.com/watch?v=7Sv4NmvdHcw

2. Generate the Fibonacci Sequence With Python
   https://www.youtube.com/watch?v=I_Giq4-2Pn8

3. Fibonacci Series in Python
   https://www.youtube.com/watch?v=Ib1bSQdXBZ0

4. Fibonacci series using recursive function in Python
   https://www.youtube.com/watch?v=Es5mHeflNCA

5. Python program to print fibonacci series upto n terms using for loop
   https://www.youtube.com/watch?v=K9Qr8bL8_UU